<a href="https://colab.research.google.com/github/gnitnaw/Programs/blob/master/Chapter3/SCO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python --version

Python 3.7.11


# Example 3.5
## Import modules

In [2]:
import numpy as np
import math
import matplotlib.pyplot as plt
print(np.__version__)

1.19.5


## Set random seed

In [3]:
np.random.seed(120)

# Example 3.17
## Input

In [4]:
c = np.array([ -6.089, -17.164, -34.054,  -5.914, -24.721,
              -14.986, -24.100, -10.708, -26.662, -22.179])

nDim = 10
#xStart = np.array([0.04067247,0.14765159,0.78323637,0.00141368,0.48526222,
#                   0.00069291,0.02736897,0.01794290,0.03729653,0.09685870])

tree = []

def addBranch(x):
  tree.append(x)

def S(x):
  y = 0
  xSum = np.sum(x)
  for i in range(nDim):
    y = y + x[i]*(c[i]+math.log(x[i]/xSum))
  return y

def constraint_1(x): #x[0]
  return 2-2*x[1]-2*x[2]-x[5]-x[9]

def constraint_2(x): #x[3]
  return 1-2*x[4]-x[5]-x[6]

def constraint_3(x): #x[7]
  return 1-x[2]-x[6]-2*x[8]-x[9]

def constraint_4(x):
  for i in range(nDim):
    if x[i] < 0.000001:
      return False
  return True

def dimension_reduce(x):
  y = x.copy()
  y[0] = constraint_1(x)
  y[3] = constraint_2(x)
  y[7] = constraint_3(x)
  return y

def penalty(x):
  y = 0
  for i in range(nDim):
    y = y + max(-(x[i]-0.000001), 0)
  y = y + S(x)
  return y

def generate_Start() :
  xStart = dimension_reduce(np.random.rand(nDim))
  while constraint_4(xStart) == False:
    xStart = dimension_reduce(np.random.rand(nDim))
  return xStart

def sampling_level1(n) : 
  x = np.zeros((n, nDim))
  for i in range(n):
    x[i] = generate_Start()
  return x

def sampling_level2(n, initValue):
  x = np.random.random((n, nDim))*0.1 + initValue
  return x

N = 100
rho = 0.5
omega = 0.5
debugMode = False
MaxTry = 100
tMax = 300
#x = xStart
#print(x)
#print(penalty(x))


## Step 1 : Generate X1...XN via uniform sampling. Set t = 0 and N_e = ceil(N rho)

In [5]:
#XGet = np.array([0.75379437, 0.48226277, 0.55370205, 0.01726460, 0.29904536, 
#                 0.17542843, 0.20921625, 0.06626207, 0.07727083, 0.01627797])
XGet = np.array([0.04067247,0.14765159,0.78323637,0.00141368,0.48526222,
                   0.00069291,0.02736897,0.01794290,0.03729653,0.09685870])
print(S(XGet))

XGet = np.array([1.77540106e+00, 8.95871298e-02, 9.49466098e-01, 5.54672599e-02, 4.37438025e-01,
                 2.80431617e-02, 4.16135281e-02, 2.47127879e-04, 3.85182508e-03, 9.69596179e-04])
print(S(XGet))

Xt = sampling_level1(N)
#Xt = sampling_level2(N, 0.1)
t = 0
N_elite = math.ceil(N*rho)
stopLoop = False
if debugMode :
  print(Xt[0:10])

SX = np.empty(N)
for i in range(N):
  SX[i] = penalty(Xt[i])

eliteXindex = np.argsort(SX)

-47.761090811433775
-61.52523707787246


## 1st while loop (Step 2)

In [6]:
while stopLoop == False :
  
  SX = SX[eliteXindex]
  Xt = Xt[eliteXindex]
  
  # Store elite X

  # Refresh S_best and X_best (Step 3)
  S_best = SX[0]
  X_best = Xt[0]

  # Step 4 : Draw Bi
  B_elite = np.random.binomial(1, 0.5, N_elite)

  ## 2nd loop, 1st for loop (Step. 5)
  RS = math.ceil(N*1.0/N_elite)
  for i in range(N_elite):
    R = B_elite[i] + RS  ## Step 6.
    if R > N_elite :
      R = N_elite
    Y = Xt[i].copy()  ## Step 7
    
    for j in range(R) : ## Step 8
      Y_p = Y.copy()
      I = np.random.randint(N_elite, size=N_elite) ## Step 9
      for k in range(N_elite):
        while I[k] == i :
          I[k] = np.random.randint(N_elite, size=1).item()
      if debugMode :
        print("i = ", i, " I[j] = ", I[j])
      SigmaX = omega*np.linalg.norm(Xt[i]-Xt[I[j]])
      Pi = np.arange(nDim)
      np.random.shuffle(Pi)  ## Step 10
      if debugMode :
        print(i, ": ", j, ": ", I, " :", SigmaX)
      for k in range(nDim): ## Step 11
        for Try in range(MaxTry):  ## Step 12
          if Pi[k] != 0 and Pi[k] != 3 and Pi[k] != 7 :
            Y_pp = Y_p.copy()
            Y_pp[Pi[k]] = Y_p[Pi[k]] + SigmaX*np.random.randn()  ## Step 13
            Y_pp = dimension_reduce(Y_pp)
            if Y_pp[Pi[k]] > 10**(-6) and Y_pp[0] > 10**(-6) and Y_pp[3] > 10**(-6) and Y_pp[7] > 10**(-6) :
              Y_p = Y_pp.copy()
              PPY = penalty(Y_p)
              if PPY < penalty(Y) :  ## Step 14
                Y = Y_p.copy()
                addBranch([t, i, j, k, Y, PPY])
                break
        Xt[i] = Y.copy()    ## Step 15
        SX[i] = penalty(Xt[i])
  eliteXindex = np.argsort(SX)
  t = t+1
  if (t%10 == 0) : 
    print("t = ", t, " Best S = ", S_best, " X = ", X_best)
  
  if t>=tMax :
    stopLoop = True


print("=========================================")
print(tree)

if debugMode :
  print(SX)
  print(eliteXindex)
  print(B_elite)

t =  10  Best S =  -47.74929422229967  X =  [0.04276243 0.15260401 0.78401354 0.00597566 0.48171707 0.00467029
 0.02591992 0.02054515 0.04509461 0.07933218]
t =  20  Best S =  -47.75924722022334  X =  [0.03950745 0.14967589 0.7841875  0.00366019 0.48466098 0.00133952
 0.02567833 0.0184157  0.04014611 0.09142626]
t =  30  Best S =  -47.760811588081  X =  [0.04062338 0.14766166 0.78494886 0.00199082 0.48474215 0.000901
 0.02762387 0.01671643 0.03872813 0.09325459]
t =  50  Best S =  -47.76102065203287  X =  [0.0405743  0.14695118 0.78498057 0.00145779 0.48474215 0.0008582
 0.02819971 0.01784415 0.03713579 0.09470398]
t =  60  Best S =  -47.761031573942844  X =  [0.04052676 0.14695118 0.78498057 0.00145779 0.48474215 0.0008582
 0.02819971 0.01779661 0.03713579 0.09475152]
t =  70  Best S =  -47.76103245164739  X =  [0.04075728 0.14695118 0.78486532 0.00145779 0.48474215 0.0008582
 0.02819971 0.01791187 0.03713579 0.09475152]
t =  80  Best S =  -47.76103735291415  X =  [0.04053121 0.146951

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

